<a href="https://colab.research.google.com/github/nmuzyka2004/Pandas-BS.../blob/main/3_actividad_web_scraping_Natalia_Muzyka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping

In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup as bs
import requests

## 🎈 Ejercicio: Arkansas 🎈


Queremos obtener de la página de Wikipedia de "Arkansas", https://es.wikipedia.org/wiki/Arkansas, la tabla que nos indica la composición étnica a aprtir del censo de EEUU de 2020.

- Revisar la página web de Wikipedia y obtener la tabla en un df
- Sobre el df: revisar qué tipo de columnas tenemos y que valores contienen las mismas
- Renombrar las columnas a nombres más descriptivos
- Eliminar columnas que no sean necesarias
- Cambiar a tipo numérico (realizando los cambios necesarios, %) las columnas que contienen números
- **Realizar un análisis exploratorio sobre la tabla**

In [47]:
# Revisar la página web de Wikipedia y obtener la tabla en un df
url = 'https://es.wikipedia.org/wiki/Arkansas'
tabla = pd.read_html(url, match = 'Composición étnica')
df = tabla[0]
df

,Raza y etnia[11]​,Alone,Alone.1,Total,Total.1
0,Blanco no hispano,68.5%,NaN,73.2%,NaN
1,Negros,14.9%,NaN,16.2%,NaN
2,Hispanios o Latinos[12]​,—,NaN,8.5%,NaN
3,Asiáticos,1.7%,NaN,2.2%,NaN
4,Nativos,0.7%,NaN,3.4%,NaN
5,Estadounidenses de las islas del Pacífico,0.5%,NaN,0.6%,NaN
6,Otro,0.3%,NaN,1.1%,NaN


In [48]:
# Sobre el df: revisar qué tipo de columnas tenemos y que valores contienen las mismas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Raza y etnia[11]​  7 non-null      object 
 1   Alone              7 non-null      object 
 2   Alone.1            0 non-null      float64
 3   Total              7 non-null      object 
 4   Total.1            0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 412.0+ bytes


In [49]:
df.columns

Index(['Raza y etnia[11]​', 'Alone', 'Alone.1', 'Total', 'Total.1'], dtype='object')

In [50]:
# Podemos renombrar las columnas para mayor accesibilidad y legibilidad
columnas = ['Raza y etnia', 'De raza unica,%', 'De raza unica histograma', 'Total (incluye mezcla racial),%', 'Total (incluye mezcla racial) histograma']

df.columns = columnas
df.head()

,Raza y etnia,"De raza unica,%",De raza unica histograma,"Total (incluye mezcla racial),%",Total (incluye mezcla racial) histograma
0,Blanco no hispano,68.5%,NaN,73.2%,NaN
1,Negros,14.9%,NaN,16.2%,NaN
2,Hispanios o Latinos[12]​,—,NaN,8.5%,NaN
3,Asiáticos,1.7%,NaN,2.2%,NaN
4,Nativos,0.7%,NaN,3.4%,NaN


In [51]:
# Eliminar columnas que no sean necesarias
df = df.drop(columns = [columna for columna in df.columns if columna.endswith("histograma")])
df

,Raza y etnia,"De raza unica,%","Total (incluye mezcla racial),%"
0,Blanco no hispano,68.5%,73.2%
1,Negros,14.9%,16.2%
2,Hispanios o Latinos[12]​,—,8.5%
3,Asiáticos,1.7%,2.2%
4,Nativos,0.7%,3.4%
5,Estadounidenses de las islas del Pacífico,0.5%,0.6%
6,Otro,0.3%,1.1%


In [52]:
# Eliminamos los corchetes
df = df.replace(r"\[\d+\]", "", regex=True)
df

,Raza y etnia,"De raza unica,%","Total (incluye mezcla racial),%"
0,Blanco no hispano,68.5%,73.2%
1,Negros,14.9%,16.2%
2,Hispanios o Latinos​,—,8.5%
3,Asiáticos,1.7%,2.2%
4,Nativos,0.7%,3.4%
5,Estadounidenses de las islas del Pacífico,0.5%,0.6%
6,Otro,0.3%,1.1%


In [53]:
# Eliminamos el simbolo de %
df = df.replace({'%': ''}, regex=True)
df

,Raza y etnia,"De raza unica,%","Total (incluye mezcla racial),%"
0,Blanco no hispano,68.5,73.2
1,Negros,14.9,16.2
2,Hispanios o Latinos​,—,8.5
3,Asiáticos,1.7,2.2
4,Nativos,0.7,3.4
5,Estadounidenses de las islas del Pacífico,0.5,0.6
6,Otro,0.3,1.1


In [54]:
# Extraemos todas las columnas
df.columns

Index(['Raza y etnia', 'De raza unica,%', 'Total (incluye mezcla racial),%'], dtype='object')

In [55]:
# La columna 'Raza y etnia' no es necesaria convertir, la excluimos haciendo slicing
columnas = list(df.columns)
columnas_a_transformar = columnas[ 1: ] #seleccionamos las columnas apartir de la posición 1
columnas_a_transformar

['De raza unica,%', 'Total (incluye mezcla racial),%']

In [56]:
#Intentamos aplicar el método pd.to_numeric
for columna in columnas_a_transformar:
  df[[columna]] = df[[columna]].apply(pd.to_numeric, errors='coerce')

In [57]:
# Convertimos a numerico todas las columnas
# Rellenamos con nulos los que no se logren convertir a numericos
df[columnas_a_transformar] = df[columnas_a_transformar].apply(pd.to_numeric, errors='coerce')
df

,Raza y etnia,"De raza unica,%","Total (incluye mezcla racial),%"
0,Blanco no hispano,68.5,73.2
1,Negros,14.9,16.2
2,Hispanios o Latinos​,NaN,8.5
3,Asiáticos,1.7,2.2
4,Nativos,0.7,3.4
5,Estadounidenses de las islas del Pacífico,0.5,0.6
6,Otro,0.3,1.1


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Raza y etnia                     7 non-null      object 
 1   De raza unica,%                  6 non-null      float64
 2   Total (incluye mezcla racial),%  7 non-null      float64
dtypes: float64(2), object(1)
memory usage: 300.0+ bytes


##Realizamos un análisis exploratorio sobre la tabla##
La tabla muestra la distribución racial y étnica de Arkansas y contiene dos columnas con valores de porcentaje. La primera es de personas que se identifican con una sola raza. Y la segunda es de porcentaje total, incluyendo personas de origen racial mixto.

In [60]:
# Revisemos si hay valores faltantes
print(df.isnull().sum())

Raza y etnia                       0
De raza unica,%                    1
Total (incluye mezcla racial),%    0
dtype: int64


Solo hay un valor faltante: en la columna "De raza unica,%" en la fila Hispanos o Latinos.

In [61]:
df.sum()

,0
Raza y etnia,Blanco no hispanoNegrosHispanios o Latinos​Asi...
"De raza unica,%",86.6
"Total (incluye mezcla racial),%",105.2


El indicador del 105.2% en el registro Total muestra que una parte significativa de la población se identifica con varias razas al mismo tiempo, por lo que algunas personas se cuentan en múltiples grupos simultáneamente. En otras palabras, el 105.2% no es un error, sino una representación del doble conteo. El 86% en la columna "De raza única, %" puede ser el resultado de la falta de datos sobre "Hispanos o Latinos" (NaN).

"Blanco no hispano" es el grupo más numeroso, con 73.2% del total.
El segundo grupo más representado son los "Negros" (16.2%).
"Hispanos o Latinos" tienen un valor no disponible (NaN) en la columna “De raza única”, pero aparecen con 8.5% en el total.
Otros grupos como Asiáticos, Nativos, Isleños del Pacífico y Otros representan juntos menos del 8% del total.

El valor NaN en “De raza única” para "Hispanos o Latinos" puede indicar que este grupo puede estar compuesto por personas de múltiples razas, y por eso no se desglosa en esa columna.

## 🎈 Ejercicio pandas 🎈


- Pandas: scrapear la tabla que contiene informacion sobre los álbums más vendidos por año en todo el mundo (https://es.wikipedia.org/wiki/Anexo:%C3%81lbumes_musicales_m%C3%A1s_vendidos) generando un dataframe con la información del año, nombre del álbum, artista y ventas. Ésta ultima columna deberá tener sus valores en numeros enteros en millones.

In [102]:
tabla = pd.read_html('https://es.wikipedia.org/wiki/Anexo:%C3%81lbumes_musicales_m%C3%A1s_vendidos', match="Artista(s)")
df = tabla[0]
df

,Año,Álbum,Artista(s),Ventas (millones),Ref(s)
0,2001,Hybrid Theory,Linkin Park,9.00,[191]​
1,2002,The Eminem Show,Eminem,13.90,[192]​
2,2003,Come Away with Me,Norah Jones,11.00,[193]​
3,2004,Confessions,Usher,12.00,[194]​
4,2005,X&Y,Coldplay,8.30,[195]​
5,2006,High School Musical,Artistas varios,7.00,[196]​
6,2007,High School Musical 2,Artistas varios,6.00,[197]​
7,2008,Viva la Vida or Death and All His Friends,Coldplay,6.80,[198]​
8,2009,I Dreamed a Dream,Susan Boyle,8.30,[199]​
9,2010,Recovery,Eminem,5.70,[200]​


In [103]:
df.columns

Index(['Año', 'Álbum', 'Artista(s)', 'Ventas (millones)', 'Ref(s)'], dtype='object')

In [104]:
# Eliminamos la última columna
df = df.drop(columns = 'Ref(s)', axis=1)
df.head()

,Año,Álbum,Artista(s),Ventas (millones)
0,2001,Hybrid Theory,Linkin Park,9.0
1,2002,The Eminem Show,Eminem,13.9
2,2003,Come Away with Me,Norah Jones,11.0
3,2004,Confessions,Usher,12.0
4,2005,X&Y,Coldplay,8.3


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Año                22 non-null     int64  
 1   Álbum              22 non-null     object 
 2   Artista(s)         22 non-null     object 
 3   Ventas (millones)  22 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 836.0+ bytes


In [106]:
# Multiplicamos ventas por 1000000
df['Ventas (millones)'] = df['Ventas (millones)'] * 1000000
df.head()

,Año,Álbum,Artista(s),Ventas (millones)
0,2001,Hybrid Theory,Linkin Park,9000000.0
1,2002,The Eminem Show,Eminem,13900000.0
2,2003,Come Away with Me,Norah Jones,11000000.0
3,2004,Confessions,Usher,12000000.0
4,2005,X&Y,Coldplay,8300000.0


In [107]:
df["Ventas (millones)"] = df["Ventas (millones)"].astype(int)
df.head()

,Año,Álbum,Artista(s),Ventas (millones)
0,2001,Hybrid Theory,Linkin Park,9000000
1,2002,The Eminem Show,Eminem,13900000
2,2003,Come Away with Me,Norah Jones,11000000
3,2004,Confessions,Usher,12000000
4,2005,X&Y,Coldplay,8300000


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Año         22 non-null     int64 
 1   Álbum       22 non-null     object
 2   Artista(s)  22 non-null     object
 3   Ventas      22 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 836.0+ bytes


In [109]:
# Renombranos la última columna
df.rename(columns={df.columns[-1]: "Ventas"}, inplace=True)
df.head()

,Año,Álbum,Artista(s),Ventas
0,2001,Hybrid Theory,Linkin Park,9000000
1,2002,The Eminem Show,Eminem,13900000
2,2003,Come Away with Me,Norah Jones,11000000
3,2004,Confessions,Usher,12000000
4,2005,X&Y,Coldplay,8300000


## 🎈 Ejercicio beautiful soup🎈


- Beautiful Soup: scrapear la tabla de lenguajes de programación más utilizados (https://www.tiobe.com/tiobe-index/) creando un dataframe con una columna llamada "lenguaje" y otra columna llamada "porcentaje_uso" (cogiendo la info de programming language y ratings respectivamente).

In [7]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# Paso 1: Obtenemos la página
url = "https://www.tiobe.com/tiobe-index/"
response = requests.get(url)
soup = bs(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>TIOBE Index - TIOBE</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.tiobe.com/wp-content/themes/tiobe/style.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="https://www.tiobe.com/xmlrpc.php" rel="pingback"/>
<link href="https://www.tiobe.com/wp-content/themes/tiobe/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://www.tiobe.com/wp-content/themes/tiobe/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://www.tiobe.com/wp-content/themes/tiobe/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="https://www.tiobe.com/wp-content/themes/tiobe/favicon/site.webmanifest" rel="manifest"/>
<link color="#287ac8" href="https://www.tiobe.com/wp-content/themes/tiobe/favicon/safari-pinned-tab.sv

In [10]:
# Paso 2: Buscamos la tabla correcta (es la primera con id "top20")
tabla = soup.find('table', id="top20")
tabla

<table class="table table-striped table-top20" id="top20">
<thead><tr>
<th style="width: 15%">Mar 2025</th>
<th style="width: 15%">Mar 2024</th>
<th style="width: 15%" title="Difference compared to last year">Change</th>
<th colspan="2" style="width: 25%">Programming Language</th>
<th style="width: 15%">Ratings</th>
<th style="width: 15%" title="Difference compared to last year">Change</th>
</tr></thead>
<tbody>
<tr><td>1</td><td>1</td><td></td><td class="td-top20"><img alt="Python page" src="/wp-content/themes/tiobe/tiobe-index/images/Python.png" style="vertical-align:middle"/></td><td>Python</td><td>23.85%</td><td>+8.22%</td></tr><tr><td>2</td><td>3</td><td><img alt="change" src="/wp-content/themes/tiobe/tpci/images/up.png"/></td><td class="td-top20"><img alt="C++ page" src="/wp-content/themes/tiobe/tiobe-index/images/C__.png" style="vertical-align:middle"/></td><td>C++</td><td>11.08%</td><td>+0.37%</td></tr><tr><td>3</td><td>4</td><td><img alt="change" src="/wp-content/themes/tiobe/

In [12]:
# Paso 3: Extraemos las filas
filas = tabla.find_all('tr')[1:]  # Saltamos el encabezado
filas

[<tr><td>1</td><td>1</td><td></td><td class="td-top20"><img alt="Python page" src="/wp-content/themes/tiobe/tiobe-index/images/Python.png" style="vertical-align:middle"/></td><td>Python</td><td>23.85%</td><td>+8.22%</td></tr>,
 <tr><td>2</td><td>3</td><td><img alt="change" src="/wp-content/themes/tiobe/tpci/images/up.png"/></td><td class="td-top20"><img alt="C++ page" src="/wp-content/themes/tiobe/tiobe-index/images/C__.png" style="vertical-align:middle"/></td><td>C++</td><td>11.08%</td><td>+0.37%</td></tr>,
 <tr><td>3</td><td>4</td><td><img alt="change" src="/wp-content/themes/tiobe/tpci/images/up.png"/></td><td class="td-top20"><img alt="Java page" src="/wp-content/themes/tiobe/tiobe-index/images/Java.png" style="vertical-align:middle"/></td><td>Java</td><td>10.36%</td><td>+1.41%</td></tr>,
 <tr><td>4</td><td>2</td><td><img alt="change" src="/wp-content/themes/tiobe/tpci/images/down.png"/></td><td class="td-top20"><img alt="C page" src="/wp-content/themes/tiobe/tiobe-index/images/C.p

In [25]:
# Paso 4: Cuantas columnas tenemos
columnas = filas[1].find_all('td')
columnas

[<td>2</td>,
 <td>3</td>,
 <td><img alt="change" src="/wp-content/themes/tiobe/tpci/images/up.png"/></td>,
 <td class="td-top20"><img alt="C++ page" src="/wp-content/themes/tiobe/tiobe-index/images/C__.png" style="vertical-align:middle"/></td>,
 <td>C++</td>,
 <td>11.08%</td>,
 <td>+0.37%</td>]

In [28]:
# Paso 4: Creamos un DataFrame
lenguajes = []
porcentajes = []
for fila in filas:
    columnas = fila.find_all('td')
    lenguaje = columnas[4].text.strip()
    porcentaje = columnas[5].text.strip()
    lenguajes.append(lenguaje)
    porcentajes.append(porcentaje)

df = pd.DataFrame({
    'lenguaje': lenguajes,
    'porcentaje_uso': porcentajes
})

df.head()

,lenguaje,porcentaje_uso
0,Python,23.85%
1,C++,11.08%
2,Java,10.36%
3,C,9.53%
4,C#,4.87%


In [29]:
# Quitamos el simbolo %
df['porcentaje_uso'] = df['porcentaje_uso'].str.replace('%', '').str.replace(',', '.').astype(float)
df.head()

,lenguaje,porcentaje_uso
0,Python,23.85
1,C++,11.08
2,Java,10.36
3,C,9.53
4,C#,4.87


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   lenguaje        20 non-null     object 
 1   porcentaje_uso  20 non-null     float64
dtypes: float64(1), object(1)
memory usage: 452.0+ bytes
